In [2]:
from tensorflow.keras.models import clone_model
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocessor
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_v2_preprocessor

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50V2

from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image


from IPython.display import clear_output


models_dict = {}
vgg16_dict, inception_v3_dict, resnet50_dict = {} ,{} ,{}


print("Loading VGG16")
model = VGG16(weights='imagenet')
model = Model(model.input, model.layers[-2].output)
vgg16_dict["model"] = clone_model(model)
vgg16_dict["preprocesor"] = vgg16_preprocessor
vgg16_dict["target_size"] = model.input_shape[1],model.input_shape[2]


print("Loading Inception v3")
model = InceptionV3(weights='imagenet')
model = Model(model.input, model.layers[-2].output)
inception_v3_dict["model"] = clone_model(model)
inception_v3_dict["preprocesor"] = inception_v3_preprocessor
inception_v3_dict["target_size"] = model.input_shape[1],model.input_shape[2]


print("Loading ResNet 50")
model = ResNet50V2(weights='imagenet')
model = Model(model.input, model.layers[-2].output)
resnet50_dict["model"] = clone_model(model)
resnet50_dict["preprocesor"] = resnet_v2_preprocessor
resnet50_dict["target_size"] = model.input_shape[1],model.input_shape[2]


models_dict["VGG16"] = vgg16_dict
models_dict["InceptionV3"] = inception_v3_dict
models_dict["ResNet50"] = resnet50_dict

clear_output()

In [25]:
import os
import cv2
import numpy as np
from keras.preprocessing import image
from skimage.io import imsave
import uuid

import matplotlib.pyplot as plt

def extract_features(image_path,model_name):
    model_dict = models_dict[model_name]
    model = model_dict["model"]
    preprocessor = model_dict["preprocesor"]
    target_size = model_dict["target_size"]
    img = cv2.resize(cv2.imread(image_path),target_size)

    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocessor(img_data)
    features = model.predict(img_data)
    return features[0]

Using TensorFlow backend.


In [13]:
def get_paths(directory):
    '''
    Devuelve la ruta relativa de las muestras .bmp
    '''
    paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".bmp"):
                paths.append(os.path.join(root, file))
    return paths

In [29]:
images = get_paths("CASIA-Polar")

In [34]:
for model_name in models_dict:
    print(model_name)

VGG16
InceptionV3
ResNet50


In [30]:
# features con modelo VGG16
feat_16 = extract_features(images[0],"VGG16")
print(feat_16.shape, feat_16[:10])

(4096,) [0.1927437  0.         0.         0.         0.12239139 0.
 0.         0.15672708 0.         0.        ]


In [31]:
# features con modelo InceptionV3
feat_inceptionV3 = extract_features(images[0],"InceptionV3")
print(feat_inceptionV3.shape, feat_inceptionV3[:10])

(2048,) [9.3350082e-04 8.3796936e-04 1.1499911e-03 5.6808785e-05 2.4289370e-03
 0.0000000e+00 8.8080979e-04 1.0621181e-03 0.0000000e+00 8.3383307e-04]


In [32]:
# features con modelo ResNet
feat_r50 = extract_features(images[0],"ResNet50")
print(feat_r50.shape, feat_r50[:10])

(2048,) [2.2836272e-02 4.1147177e-05 1.0470141e-02 4.1030847e-02 5.5118497e-05
 4.9771946e-03 5.8555678e-02 1.9911148e-02 1.5799802e-02 2.6991161e-02]


In [53]:
import pandas as pd

df = pd.read_csv("iris_data.csv", index_col=0) # quitamos columna unnamed
df.rename(columns={'image':'Image','class':'Clase'}, inplace=True)
df.head()

,Image,pupil x_center,pupil y_center,pupil radius,iris x_center,iris y_center,iris radius,Clase
0,Adrianna_1.bmp,182,134,36,177,134,102,Adrianna
1,Adrianna_2.bmp,174,138,38,172,138,38,Adrianna
2,Adrianna_3.bmp,173,120,38,167,120,104,Adrianna
3,Adrianna_4.bmp,183,122,40,185,122,38,Adrianna
4,Adrianna_5.bmp,177,145,38,179,143,36,Adrianna


In [57]:
def register_to_deepfeatures(register):
    """
    Compute all CNN features    
    
    Parameters
    ----------
    register : Series
        Serie containing metadata of the image
   
    Returns
    -------
    results : Series 
        A Pandas Serie contanining all of the features

    """
    img_name = register.Image
    clase = register.Clase # clase   
    
    img_dir = os.path.join("CASIA-Polar",clase)
    img_path = os.path.join(img_dir,img_name)

#     img = imread(img_path)

    basic_values = pd.Series([img_name,clase],["Name","Class"])

    all_features = []
    all_names = []

    for model_name in models_dict.keys():
        features = extract_features(img_path,model_name)
        names = [f"{model_name}_{i}" for i in range(len(features))]
        all_features+=list(features)
        all_names+= names
        results = pd.Series(all_features,all_names)

    return pd.concat((basic_values,results))


In [58]:
example1 = df.iloc[5]
example1

Image             Adrianna_6.bmp
pupil x_center               178
pupil y_center               134
pupil radius                  38
iris x_center                179
iris y_center                132
iris radius                   40
Clase                   Adrianna
Name: 5, dtype: object

In [59]:
results = register_to_deepfeatures(example1)

In [64]:
results

Name             Adrianna_6.bmp
Class                  Adrianna
VGG16_0                0.250871
VGG16_1               0.0172912
VGG16_2                       0
VGG16_3                       0
VGG16_4               0.0870106
VGG16_5                       0
VGG16_6                       0
VGG16_7                0.093244
VGG16_8                       0
VGG16_9                       0
VGG16_10              0.0121606
VGG16_11                      0
VGG16_12              0.0823187
VGG16_13               0.182892
VGG16_14                      0
VGG16_15               0.265486
VGG16_16               0.316148
VGG16_17                      0
VGG16_18              0.0902662
VGG16_19                      0
VGG16_20                      0
VGG16_21               0.304199
VGG16_22              0.0050865
VGG16_23               0.274025
VGG16_24                      0
VGG16_25                      0
VGG16_26                      0
VGG16_27                      0
                      ...      
ResNet50

In [65]:
df_deep = df.apply(register_to_deepfeatures, axis=1)

In [66]:
df_deep

,Name,Class,VGG16_0,VGG16_1,VGG16_2,VGG16_3,VGG16_4,VGG16_5,VGG16_6,VGG16_7,...,ResNet50_2038,ResNet50_2039,ResNet50_2040,ResNet50_2041,ResNet50_2042,ResNet50_2043,ResNet50_2044,ResNet50_2045,ResNet50_2046,ResNet50_2047
0,Adrianna_1.bmp,Adrianna,0.192744,0.000000,0.000000,0.000000,0.122391,0.0,0.000000,0.156727,...,0.012919,0.002492,0.017621,0.031463,0.030044,0.005256,0.028403,0.024358,0.018938,0.011701
1,Adrianna_2.bmp,Adrianna,0.247441,0.000000,0.000000,0.000000,0.111527,0.0,0.000000,0.166326,...,0.013752,0.002917,0.016202,0.030415,0.031504,0.006511,0.026331,0.022146,0.017631,0.011405
2,Adrianna_3.bmp,Adrianna,0.245851,0.015115,0.000000,0.000000,0.074128,0.0,0.000000,0.095503,...,0.011168,0.004267,0.019801,0.031529,0.031478,0.006267,0.026642,0.020293,0.019257,0.010068
3,Adrianna_4.bmp,Adrianna,0.257767,0.000365,0.000000,0.000000,0.081624,0.0,0.000000,0.184155,...,0.013856,0.002902,0.018040,0.032082,0.032178,0.005347,0.026247,0.022378,0.017346,0.012369
4,Adrianna_5.bmp,Adrianna,0.209670,0.011240,0.000000,0.000000,0.046138,0.0,0.000000,0.186705,...,0.014179,0.002009,0.017692,0.029324,0.027715,0.006873,0.029411,0.025534,0.020655,0.012881
5,Adrianna_6.bmp,Adrianna,0.250871,0.017291,0.000000,0.000000,0.087011,0.0,0.000000,0.093244,...,0.011844,0.002739,0.018474,0.033087,0.031854,0.004274,0.025800,0.023644,0.019558,0.011827
6,Adrianna_7.bmp,Adrianna,0.216848,0.071096,0.000000,0.000000,0.045389,0.0,0.000000,0.125871,...,0.012276,0.002071,0.016979,0.030182,0.029912,0.005857,0.030023,0.023708,0.018060,0.011848
7,Aharon_1.bmp,Aharon,0.244032,0.001368,0.026120,0.000000,0.069160,0.0,0.000000,0.038188,...,0.011621,0.003633,0.018256,0.033586,0.031060,0.003396,0.024546,0.021225,0.018326,0.010921
8,Aharon_2.bmp,Aharon,0.240948,0.000000,0.016262,0.000000,0.085209,0.0,0.000000,0.072026,...,0.010831,0.004694,0.019990,0.033268,0.026910,0.003160,0.024732,0.025172,0.016801,0.013645
9,Aharon_3.bmp,Aharon,0.257789,0.000000,0.053026,0.000000,0.152313,0.0,0.000000,0.032832,...,0.012191,0.002373,0.015293,0.033343,0.028428,0.005295,0.022888,0.025981,0.016851,0.014523


In [67]:
df.to_csv("iris_deep_features.csv")